# Métodos para Resolução de Sistemas Lineares #

Nesta aula, iremos implementar métodos para encontrar vetores de solução $x$ para sistemas lineares no formato $Ax=b$.

# Preliminares #

### Resolvendo sistemas triangulares ###

Normalmente, para resolver sistemas lineares, precisamos chegar de alguma forma a sistemas triangulares (inferiores ou superiores). Por que? Porque eles podem ser resolvidos pelo método das substituições retroativas.



$$ x_{i} = \frac{b_{i}-\sum\limits_{j=1}^{i-1}{a_{ij}x_{j}}}{a_{ii}} $$

e

$$ x_{i} = \frac{b_{i}-\sum\limits_{j=i+1}^{n}{a_{ij}x_{j}}}{a_{ii}} $$

Faça duas funções, que recebendo como parâmetro uma matriz $A$ triangular (superior ou inferior) e um vetor $b$, retorne o vetor solução $X$:

In [1]:
import numpy as np

In [2]:
def resolveTS(A,b): #A Triangular Superior
    x= np.zeros(len(A))
    for i in range(len(A)-1, -1,-1):
        x[i]=(b[i]-(A[i,i+1:]*x[i+1:]).sum())/A[i,i]
    return x

def resolveTI(A,b): #A Triangular Inferior
    m = len(A)
    n = len(A[0])
    k=0
    x = np.zeros(n)
    for i in range(0,m):
        k=0
        for j in range(0,n-1):
            k = A[i][j] * x[j]
        x[i] = (b[i]-k)/A[i][i]
        
    return x


In [3]:
Mt = np.array([[1,1,-2,0],
               [0,4,0,1],
               [0,0,3,7/4],
               [0,0,0,-5/3]])
Bt = np.array([5,11,9/4, -5])
resolveTS(Mt, Bt)

array([ 1.,  2., -1.,  3.])

In [4]:
Mti = np.flip(Mt)
Bti = np.flip(Bt)

print(Mti)
print(Bti)

[[-1.66666667  0.          0.          0.        ]
 [ 1.75        3.          0.          0.        ]
 [ 1.          0.          4.          0.        ]
 [ 0.         -2.          1.          1.        ]]
[-5.    2.25 11.    5.  ]


In [5]:
resolveTI(Mti,Bti)

array([3.  , 0.75, 2.75, 2.25])

### Operações $l$-elementares ###

Fazer operações $l$-elementares com matrizes numpy é bem simples: com `M[i:]` você acessa a i-ésima linha:

In [6]:
M = np.random.randint(-10,10,(4,4))
print(M)

print("Subtraindo uma linha por outra multiplicada por uma constante:")
M[2] = M[2] - M[1]*2
print(M)

print("Trocar duas linhas:")

M[[1,3]] = M[[3,1]]
print(M)

[[ -6  -7  -1  -8]
 [  5  -2   1  -7]
 [ -5  -1  -7  -3]
 [ -8  -1 -10   5]]
Subtraindo uma linha por outra multiplicada por uma constante:
[[ -6  -7  -1  -8]
 [  5  -2   1  -7]
 [-15   3  -9  11]
 [ -8  -1 -10   5]]
Trocar duas linhas:
[[ -6  -7  -1  -8]
 [ -8  -1 -10   5]
 [-15   3  -9  11]
 [  5  -2   1  -7]]


### Outras operações com Matrizes ###

In [7]:
print("A matriz")
print(M)
print("Achar o maior elemento da matriz:")

print(M[1].max())

print("Achar o maior elemento da matriz em módulo:")
print(abs(M[:][2]).max()) # ABS e o valor absoluto, pegar o maior das linhas na 2 coluna mdsssss


# E PARA ACHAR O MAIOR ELEMENTO DE UMA LINHA?
#para pegar os elementos da coluna 2
print("Para mostrar os elementos da 2 linha:")
print("tem essa maneira:")
print(M[2])
print("e tem dessa outra:")
print(M[:][2])
print("Para mostrar os elementos da 2 coluna:")
print(M[0])


A matriz
[[ -6  -7  -1  -8]
 [ -8  -1 -10   5]
 [-15   3  -9  11]
 [  5  -2   1  -7]]
Achar o maior elemento da matriz:
5
Achar o maior elemento da matriz em módulo:
15
Para mostrar os elementos da 2 linha:
tem essa maneira:
[-15   3  -9  11]
e tem dessa outra:
[-15   3  -9  11]
Para mostrar os elementos da 2 coluna:
[-6 -7 -1 -8]


In [8]:
print("A matriz:\n", M)
print("Matriz transposta de M:\n",M[:,:].T)

print("Determinante de M: ",np.linalg.det(M))

print("Autovalores de uma matriz:")

(a,_) = np.linalg.eig(M) 
print(a)

A matriz:
 [[ -6  -7  -1  -8]
 [ -8  -1 -10   5]
 [-15   3  -9  11]
 [  5  -2   1  -7]]
Matriz transposta de M:
 [[ -6  -8 -15   5]
 [ -7  -1   3  -2]
 [ -1 -10  -9   1]
 [ -8   5  11  -7]]
Determinante de M:  2223.000000000002
Autovalores de uma matriz:
[-16.89434118+0.j          -2.08193503+7.96423928j
  -2.08193503-7.96423928j  -1.94178876+0.j        ]


# Métodos Exatos #

## Eliminação gaussiana ##

Para fazer a eliminação gaussiana deve-se usar os elementos da diagonal principal de A como pivos para zerar os elementos da mesma coluna.

Implemente a eliminação gaussiana simples (sem pivotação parcial):

In [9]:
Mzao = np.array([[2,1],
               [1,3]])
bzao = np.array([[5],[6]])

belowD = np.triu(Mzao, -1)
aboveD = np.tril(Mzao, 1)
print(belowD)
print(aboveD)
print("achar a diagonal principal:") 
print(Mzao.diagonal())

[[2 1]
 [1 3]]
[[2 1]
 [1 3]]
achar a diagonal principal:
[2 3]


In [10]:
def eliminacaoGaussianaSimples(A,b):
    pivot = 0
    m = 0.0
    x = np.zeros(len(A))
    for i in range(0,len(A)-1):
        pivot = A[i,i]
        for j in range(i+1,len(A)):
            if (pivot == 0):
                m = 0
            else:
                m = A[j,i]/pivot
            A[j] =  A[j] - (m*A[i])
            b[j] = b[j] - (m*b[i])       
    x = resolveTS(A,b)
    return x

In [11]:
 
OtaM = np.array([[1,1,-2,1],
                 [-1,3,2,1],
                 [1,2,1,2],
                 [2,0,-2,-1]])
OtaB = np.array([5,6,10,1])
print(OtaM)

[[ 1  1 -2  1]
 [-1  3  2  1]
 [ 1  2  1  2]
 [ 2  0 -2 -1]]


In [12]:
eliminacaoGaussianaSimples(OtaM,OtaB)
#substitui(OtaM)

array([2.58333333, 1.75      , 0.66666667, 2.        ])

In [13]:

M = np.array([[1,-1,-2],
              [-1,3,-2],
              [-1,-2,1]])
bt = np.array([2,3,4])
print(M)

eliminacaoGaussianaSimples(M,bt)

[[ 1 -1 -2]
 [-1  3 -2]
 [-1 -2  1]]


array([-2.92857143, -1.21428571, -1.85714286])

## Decomposição LU ##

A decomposição LU é mais utilizada quando a mesma matriz de coeficientes $A$ é usada para várias soluções diferentes. Por isto, ela pode ser dividida em dois passos:

- Decompor $A$ em $L$ e $U$ 
- Dados $L$, $U$ e $b$, achar a solução X

Faça as duas funções, com a decomposição ainda sem pivotação parcial:

In [14]:
Matriz = np.array([[2,1],
                  [3,5]])
x = np.copy(Matriz) #copia a matriz para uma outra matriz
t = np.shape(x)[0]#tamanho de uma matriz
k = np.eye(t) # matriz identidade
print(x)
print(t)
print(k)

[[2 1]
 [3 5]]
2
[[1. 0.]
 [0. 1.]]


In [15]:

Mtz = np.array([[1,6,2,4],
               [3,19,4,15],
               [1,4,8,-12],
               [5,33,9,3]])
bt = np.array([8,25,18,72])


In [16]:
# decompor A para:
#deixar o triangulo inferior nulo para isso usar a funcao 

def decompoeLU(A):  
    U = np.copy(A)  
    n = np.shape(U)[0]  
    L = np.eye(n)  
    for j in np.arange(n-1):  
        for i in np.arange(j+1,n):  
            L[i,j] = U[i,j]/U[j,j]  
            for k in np.arange(j+1,n):  
                U[i,k] = U[i,k] - L[i,j]*U[j,k]  
            U[i,j] = 0  
    return (L, U)

l, u = decompoeLU(Mtz)
#funciona
def resolveLU(L,U,b):
    t = resolveTI(L,b)
    x = resolveTS(U,t)
    return x

In [17]:
L, U = decompoeLU(Mtz)
resolveLU(L, U, bt)

array([1244., -146., -126.,  -27.])

### Pivotação Parcial ###

Sistemas onde o determinante de uma das submatrizes principais ($A_{1x1},A_{2x2},A_{3x3}...$) é igual a $0$ não podem ser resolvidos com a decomposição LU simples. Nestes casos, deve-se utilizar a pivotação parcial, onde o pivô é escolhido da linha com o maior elemento em módulo.

Contudo, é importante guardar as trocas de linha que foram efetuadas na matriz de permutações $P$. Esta matriz é uma matriz identidade com as linhas trocadas junto com a pivotação. Por exemplo, se na primeira coluna o maior elemento está na linha três, este será o primeiro pivô (a linha 1 será trocada com a 3). Neste caso, na matriz $P$ também se troca estas linhas. No fim do processo:

$PAx = Pb$

$LUx=Pb$

Então basta resolver trocando as linhas de b de através da multiplicação por P. Lembrem que multiplicar matrizes em numpy é:

`p.dot(a)` ou `dot(p,a)`

Implemente uma função que checa se a pivotação parcial é necessária e a decomposição LU com pivotação parcial:

In [52]:
def verificaPivot(A):# escolher a maior linha em modulo
    for i in range(0,len(A)):
        if np.linalg.det(A[:i+1,:i+1]) != 0:
            return 0
    return 1

def LUparcial(A):
    P = np.eye(len(A))
    L = np.eye(len(A))
    i = 0
    for i in range(0,len(A)-1):
        im = np.argmax(abs(A[i:,i])) + i
        pivo = A[im][i]
        for j in range(i,len(A)):        
            m = A[j][i] / pivo
            if j != im: 
                A[j] = A[j] - A[im] * m
                if j < im: 
                    L[j+1][i] = m
                else: 
                    L[j][i] = m
    
        A[[i,im]]=A[[im,i]]
        P[[i,im]]=P[[im,i]]
    U = A
    return L,U,P

def resolveLUpar(L,U,P,b):
    y = resolveTI(L,np.dot(P,b))
    x = resolveTS(U, y)
    return x


In [53]:
L, U, P = LUparcial(Mtz)
resolveLUpar(L,U,P,bt)

array([-5330.3,   725.5,   294. ,    24. ])

#### Exercício ####

Rode e verifique o tempo com `%timeit` da eliminação gaussiana, decomposição LU e LU com pivotação para o sistema abaixo. No caso da LU, calcule o tempo da decomposição e da solução dos sistemas:

In [43]:
#Sistema 1:

M1 = np.array([[1,-3,5,6], [-8,4,-1,0],[3,2,-2,7],[1,2,5,-4]])
b1 = np.array([17,29,-11,7])

print("Eliminação gaussiana:", eliminacaoGaussianaSimples(M1,b1))
%timeit -n100 eliminacaoGaussianaSimples(M1,b1)

print("\n")

print("Decomposição LU:")
print("Função de decomposição:")
%timeit -n100 decompoeLU(M1)
L, U = decompoeLU(M1)
print("Função Resolve:", resolveLU(L,U,b1))
%timeit -n100 resolveLU(L,U,b1)


print("\n")
print("LU com pivotação parcial:")
%timeit -n100 LUparcial(M1)
L, U, P = LUparcial(M1)
print("Resolve LU parcial:", resolveLUpar(L,U,P,b1))
%timeit -n100 resolveLUpar(L,U,P,b1)




Eliminação gaussiana: [-3.8516129   0.32903226  3.12903226  1.03225806]
426 µs ± 27 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


Decomposição LU:
Função de decomposição:
421 µs ± 5.71 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
Função Resolve: [-3.8516129   0.32903226  3.12903226  1.03225806]
211 µs ± 16.9 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


LU com pivotação parcial:
444 µs ± 30 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
Resolve LU parcial: [-3.8516129   0.32903226  3.12903226  1.03225806]
163 µs ± 16.4 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [44]:

#Sistema 2:

M2 = np.array([[-2,3,1,5],
               [5,1,-1,0],
               [1,6,3,-1],
               [4,5,2,8]])
b2 = np.array([2,-1,0,6])
print("Eliminação gaussiana:", eliminacaoGaussianaSimples(M2,b2))
%timeit -n100 eliminacaoGaussianaSimples(M2,b2)

print("\n")

print("Decomposição LU:")
print("Função de decomposição:")
%timeit -n100 decompoeLU(M2)
L, U = decompoeLU(M2)
print("Função Resolve:", resolveLU(L,U,b2))
%timeit -n100 resolveLU(L,U,b2)


print("\n")
print("LU com pivotação parcial:")
%timeit -n100 LUparcial(M2)
L, U, P = LUparcial(M2)
print("Resolve LU parcial:", resolveLUpar(L,U,P,b2))
%timeit -n100 resolveLUpar(L,U,P,b2)


Eliminação gaussiana: [ 0.43125 -0.6125   1.7      0.6    ]
446 µs ± 39.7 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


Decomposição LU:
Função de decomposição:
429 µs ± 14.6 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
Função Resolve: [ 0.43125 -0.6125   1.7      0.6    ]
247 µs ± 29.5 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


LU com pivotação parcial:
578 µs ± 178 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
Resolve LU parcial: [ 0.43125 -0.6125   1.7      0.6    ]
276 µs ± 101 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


## Cholesky ##

O método de Cholesky só pode ser utilizado quando a matriz for:

- Simétrica (igual a sua transposta)
- Definida positiva: 
    - Todos os elementos da diagonal principal são positivos
    - Todos os autovalores de $A$ são positivos
    - Todas as submatrizes superiores possuem determinante __positivo__.

Se for possível, o método de Cholesky é uma decomposição LU onde $U=L^{T}$, ou seja $LL^{T}x=b$.

Lembrando que na decomposição de Cholesky os elementos da diagonal principal de L são:

$$l_{jj} = \sqrt{a_{jj}-\sum\limits_{k=1}^{j-1}{l_{jk}^2}, j = 1,2,...,n}$$

E os fora da diagonal principal são:

$$ l_{ij} = \frac{a_{ij}-\sum\limits_{k=1}^{j-1}{l_{ik}l_{jk}}}{l_{jj}} $$

Desta forma, faça uma função para determinar se uma matriz pode ser resolvida via Cholesky e uma para encontrar L (para resolver pode-se usar `resolveLU(L,L.T,b)`):

In [45]:


#sim, funciona
def simetrica(A):
    At = A[:,:].T
    x = np.equal(A, At)
    if(np.all(x == True)): #verificar se a A e igual a transposta de A (Simetria)
        return True

def diagonalPositiva(A):
    if(np.all(A.diagonal() > 0)): #verificar se a diagonal principal e positiva
        return True

def autoValPos(A):
    (a,_) = np.linalg.eig(A)
    for i in range(len(A)): # autovalores positivos
        if(a[i] < 0):
            return False
        else:
            return True
def subMatDetPos(A):
     for j in range(len(A)):
        t = np.linalg.det(A[0:j,0:j]) # determinantes das submatrizes positivas
        if(t < 0):
            return False
        else:
            return True      
def verificaCholesky(A):
    x = simetrica(A)
    y = diagonalPositiva(A)
    t = autoValPos(A)
    h = subMatDetPos(A)
    if ((x == True) and (y==True) and (t==True) and (h==True)):
        return True
    else:
        return False
      

def geraCholesky(A):
    L = [[0.0] * len(A) for _ in range(len(A))]
    for i, (Ai, Li) in enumerate(zip(A, L)):
        for j, Lj in enumerate(L[:i+1]):
            s = sum(Li[k] * Lj[k] for k in range(j))
            Li[j] = np.sqrt(Ai[i] - s) if (i == j) else \
                      (1.0 / Lj[j] * (Ai[j] - s))
    L = np.array(L)
    return L

def resolveCholesky(A,b):
    a = verificaCholesky(A)
    if(a == True):
        L = geraCholesky(A) #gera L
        x = resolveLU(L,L.T,b) # resolve L e a transposta de L
        return x
    else:
        return -1
        

In [46]:
M3 = np.array([[9,-6,3],
               [-6,29,-7],
               [3,-7,18]])
b3 = np.array([-3,-8,33])

verificaCholesky(M3)
resolveCholesky(M3,b3)


array([-0.93916667,  0.0725    ,  1.9625    ])

No método de Cholesky, por ser uma matriz simétrica, podemos, ao invés de calcular o determinante normalmente, usar a seguinte definição:

$$ det(A) = det(L)det(L') $$

$$ det(A) = \bigg(\prod_{i=1}^{n}{l_{ii}}\bigg)^2 $$

In [48]:
def detCholesky(A):
    L = geraCholesky(A)
    detL = np.linalg.det(L)
    detLT = np.linalg.det(L.T)
    det = detL*detLT
    return det

#### Exercicios ####

1 - Verifique se os seguintes sistemas podem ser resolvidas via Choleski

2 - Compare o tempo para decomposição LU das que são possíveis com a de Choleski usando `%timeit`

3 - Compare o tempo do calculo do determinante de numpy com o determinante específico para matrizes para choleski

In [49]:
M3 = np.array([[9,-6,3],
               [-6,29,-7],
               [3,-7,18]])
b3 = np.array([-3,-8,33])

print("Cholesky:")
print("Verifica se o sistema pode ser resolvido, senao for retorna -1:", resolveCholesky(M3,b3))
%timeit -n100 resolveCholesky(M3,b3)


print("\n")
print("Tempo da decomposição LU:")
print("tempo da função de decomposição:")
L, U = decompoeLU(M3)
%timeit -n100 decompoeLU(M3)

print("tempo da função Resolve:", resolveLU(L,U,b3))
%timeit -n100 resolveLU(L,U,b3)


print("\n")
print("Tempo do determinante numpy:", np.linalg.det(M3))
%timeit -n100 np.linalg.det(M3)

print("Tempo do determinante Cholesky:", detCholesky(M3))
%timeit -n100 detCholesky(M3)

Cholesky:
Verifica se o sistema pode ser resolvido, senao for retorna -1: [-0.93916667  0.0725      1.9625    ]
639 µs ± 70.9 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


Tempo da decomposição LU:
tempo da função de decomposição:
202 µs ± 26.5 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
tempo da função Resolve: [-0.93916667  0.0725      1.9625    ]
166 µs ± 10.9 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


Tempo do determinante numpy: 3599.999999999998
37.4 µs ± 9.9 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
Tempo do determinante Cholesky: 3599.999999999998
213 µs ± 19.5 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [50]:
M4 = np.array([[4,-2,4,10],
               [-2,2,-1,-7],
               [4,-1,14,11],
               [10,-7,11,31]])
b4 = np.array([2,2,-1,-2])
print("Cholesky:")
print("Verifica se o sistema pode ser resolvido, senao for retorna -1:", resolveCholesky(M4,b4))
%timeit -n100 resolveCholesky(M4,b4)


print("\n")
print("Tempo da decomposição LU:")
print("tempo da função de decomposição:")
L, U = decompoeLU(M4)
%timeit -n100 decompoeLU(M4)

print("tempo da função Resolve:", resolveLU(L,U,b4))
%timeit -n100 resolveLU(L,U,b4)


print("\n")
print("Tempo do determinante numpy:", np.linalg.det(M4))
%timeit -n100 np.linalg.det(M4)

print("Tempo do determinante Cholesky:", detCholesky(M4))
%timeit -n100 detCholesky(M4)

Cholesky:
Verifica se o sistema pode ser resolvido, senao for retorna -1: [ 3.33333333 -1.77777778  0.44444444 -1.66666667]
781 µs ± 86.1 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


Tempo da decomposição LU:
tempo da função de decomposição:
431 µs ± 58.2 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
tempo da função Resolve: [ 3.33333333 -1.77777778  0.44444444 -1.66666667]
221 µs ± 3.84 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


Tempo do determinante numpy: 35.999999999999964
36 µs ± 3.58 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
Tempo do determinante Cholesky: 36.0
304 µs ± 17.4 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [51]:

M5 = np.array([[16,-4,4,12],
               [-4,2,-1,-7],
               [4,-1,26,13],
               [12,-7,13,25]])
b5 = np.array([2,2,-1,-2])
print("Cholesky:")
print("Verifica se o sistema pode ser resolvido, senao for retorna -1:", resolveCholesky(M5,b5))
%timeit -n100 resolveCholesky(M5,b5)


print("\n")
print("Tempo da decomposição LU:")
print("tempo da função de decomposição:")
L, U = decompoeLU(M5)
%timeit -n100 decompoeLU(M5)

print("tempo da função Resolve:", resolveLU(L,U,b5))
%timeit -n100 resolveLU(L,U,b5)


print("\n")
print("Tempo do determinante numpy:", np.linalg.det(M5))
%timeit -n100 np.linalg.det(M5)

print("Tempo do determinante Cholesky:", detCholesky(M5))
%timeit -n100 detCholesky(M5)

Cholesky:
Verifica se o sistema pode ser resolvido, senao for retorna -1: [nan nan nan nan]


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:44: RuntimeWarning: invalid value encountered in sqrt


811 µs ± 64 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


Tempo da decomposição LU:
tempo da função de decomposição:
424 µs ± 9.21 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
tempo da função Resolve: [ 0.775  3.6   -0.2    0.4  ]
219 µs ± 15.6 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


Tempo do determinante numpy: -1599.9999999999984
38.8 µs ± 5.21 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


C:\ProgramData\Anaconda3\lib\site-packages\numpy\linalg\linalg.py:2022: RuntimeWarning: invalid value encountered in det
  r = _umath_linalg.det(a, signature=signature)


Tempo do determinante Cholesky: nan
348 µs ± 61.8 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
